# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [252]:
from pipeline import get_connection, get_cursor

## Setup

In [253]:
conn = get_connection()
cursor = get_cursor(conn)

## Queries

#### 1. What exhibition is most frequently visited?

In [254]:
cursor.execute("""
               SELECT e.exhibition_name, COUNT(rei.request_interaction_id) + COUNT(rai.rating_interaction_id) AS total_interactions
               FROM exhibition AS e 
               LEFT JOIN request_interaction AS rei ON e.exhibition_id = rei.exhibition_id
               LEFT JOIN rating_interaction AS rai ON e.exhibition_id = rai.exhibition_id
               GROUP BY e.exhibition_id
               ORDER BY total_interactions DESC
               LIMIT 1""")
cursor.fetchall()

[RealDictRow([('exhibition_name', 'Cetacean Sensations'),
              ('total_interactions', 27376)])]

#### 2. What hour of the day has the most ratings?

In [255]:
cursor.execute("""
SELECT EXTRACT(HOUR FROM event_at) AS hour, COUNT(rai.rating_interaction_id) AS rating_count
FROM rating_interaction as rai 
GROUP BY hour 
ORDER BY rating_count DESC 
LIMIT 1""")
cursor.fetchall()

[RealDictRow([('hour', Decimal('10')), ('rating_count', 232)])]

#### 3. What exhibition has the most emergencies?

In [256]:
cursor.execute("""
SELECT e.exhibition_name, COUNT(re.request_description) AS emergency_count 
FROM exhibition AS e
LEFT JOIN request_interaction AS rei ON e.exhibition_id = rei.exhibition_id 
LEFT JOIN request AS re ON rei.request_id = re.request_id 
WHERE re.request_description = 'Emergency' 
GROUP BY e.exhibition_name 
ORDER BY emergency_count DESC""")
cursor.fetchall()

[RealDictRow([('exhibition_name', 'The Crenshaw Collection'),
              ('emergency_count', 2)])]

#### 4. What is the average rating for each exhibition?

In [257]:
cursor.execute("""
SELECT e.exhibition_name, ROUND(AVG(ra.rating_value), 2) AS average_rating
FROM exhibition AS e
LEFT JOIN rating_interaction AS rai ON e.exhibition_id = rai.exhibition_id 
LEFT JOIN rating AS ra ON rai.rating_id = ra.rating_id 
GROUP BY e.exhibition_name 
ORDER BY e.exhibition_name ASC""")
cursor.fetchall()

[RealDictRow([('exhibition_name', 'Adaptation'),
              ('average_rating', Decimal('1.93'))]),
 RealDictRow([('exhibition_name', 'Cetacean Sensations'),
              ('average_rating', Decimal('2.83'))]),
 RealDictRow([('exhibition_name', 'Measureless to Man'),
              ('average_rating', Decimal('1.92'))]),
 RealDictRow([('exhibition_name', 'Our Polluted World'),
              ('average_rating', Decimal('1.22'))]),
 RealDictRow([('exhibition_name', 'The Crenshaw Collection'),
              ('average_rating', Decimal('1.43'))]),
 RealDictRow([('exhibition_name', 'Thunder Lizards'),
              ('average_rating', None)])]

#### 5. Are positive ratings more frequent before or after 1pm?

In [258]:
def determine_greater_satisfaction(before, after):
    if before>after:
        return f"There are {before-after} more positive reviews before 1pm."
    if before<after:
        return f"There are {after-before} more positive reviews after 1pm."
    return "There are equally as many positive reviews before and after 1pm."

# Before 1 pm

cursor.execute("""
SELECT COUNT(rai.rating_id) AS positive_ratings_before_1pm 
FROM rating_interaction AS rai 
LEFT JOIN rating AS ra ON rai.rating_id = ra.rating_id 
WHERE ra.rating_value >= 3 AND EXTRACT(HOUR FROM rai.event_at) >= 13""")
before = cursor.fetchall()

# After 1 pm 

cursor.execute("""
SELECT COUNT(rai.rating_id) AS positive_ratings_after_1pm 
FROM rating_interaction AS rai 
LEFT JOIN rating AS ra ON rai.rating_id = ra.rating_id 
WHERE ra.rating_value >= 3 AND EXTRACT(HOUR FROM rai.event_at) < 13""")
after = cursor.fetchall()

print(f"No. of positive ratings before 1pm = {before[0]['positive_ratings_before_1pm']}")
print(f"No. of positive ratings after 1pm = {after[0]['positive_ratings_after_1pm']}")
print(determine_greater_satisfaction(
    before[0]['positive_ratings_before_1pm'], after[0]['positive_ratings_after_1pm']))

No. of positive ratings before 1pm = 344
No. of positive ratings after 1pm = 313
There are 31 more positive reviews before 1pm.


#### 6. Do Zoology exhibitions get better ratings than other types?

In [259]:
def determine_greater_satisfaction(zoology, other):
    if zoology > other:
        return f"Zoology exhibitions have higher ratings compared to non-Zoology exhibitions."
    if zoology < other:
        return f"Non-Zoology exhibitions have higher ratings compared to Zoology exhibitions."
    return "There are equally as many positive reviews before and after 1pm."

# ratings for Zoology exhibitions


cursor.execute("""
SELECT ROUND(AVG(ra.rating_value), 2) AS average_rating 
FROM exhibition AS e 
LEFT JOIN department AS d ON e.department_id = d.department_id 
LEFT JOIN rating_interaction AS rai ON e.exhibition_id = rai.exhibition_id 
LEFT JOIN rating AS ra ON rai.rating_id = ra.rating_id 
WHERE d.department_name = 'Zoology'""")
zoology =cursor.fetchall()


# ratings for other exhibitions

cursor.execute("""
SELECT ROUND(AVG(ra.rating_value), 2) AS average_rating 
FROM exhibition AS e 
LEFT JOIN department AS d ON e.department_id = d.department_id 
LEFT JOIN rating_interaction AS rai ON e.exhibition_id = rai.exhibition_id 
LEFT JOIN rating AS ra ON rai.rating_id = ra.rating_id 
WHERE d.department_name != 'Zoology'""")
other = cursor.fetchall()

print(f"Zoology has an average rating of {zoology[0]['average_rating']} for its exhibitions.")
print(f"Other departments have an average rating of {other[0]['average_rating']} for their exhibitions.")
print(determine_greater_satisfaction(
    zoology[0]['average_rating'], other[0]['average_rating']))

Zoology has an average rating of 2.20 for its exhibitions.
Other departments have an average rating of 1.64 for their exhibitions.
Zoology exhibitions have higher ratings compared to non-Zoology exhibitions.
